In [4]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [1]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [ ]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

In [8]:
try:
    from allconv import AllConvNet
except ModuleNotFoundError:
    !wget https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
    from allconv import AllConvNet

--2023-10-24 20:31:50--  https://github.com/StefOe/all-conv-pytorch/raw/HEAD/allconv.py
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/StefOe/all-conv-pytorch/8545bb3ff7ac4410e52d8db930f0485074073ea9/allconv.py [following]
--2023-10-24 20:31:50--  https://raw.githubusercontent.com/StefOe/all-conv-pytorch/8545bb3ff7ac4410e52d8db930f0485074073ea9/allconv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1457 (1.4K) [text/plain]
Saving to: ‘allconv.py’

allconv.py          100%[===================>]   1.42K  --.-KB/s    in 0s      

2023-10-24 20:31:51 (28.7 MB/s) - ‘allconv.py’ saved [1457/1

In [11]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0, 1, 2)) / 255
train_std = trainset.data.std(axis=(0, 1, 2)) / 255
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [12]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)

In [17]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))


def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))

    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        if gsync_save:
            gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [ ]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.302415
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.302252
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.302697
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.302792
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.302699
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.303305
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.302625
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.302336
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.301642
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.304374
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.303802
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.303093
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.302794
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.302592
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.302551
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.302581

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



Uploading file best.pt: 100%|██████████| 100/100 [00:02<00:00, 47.48it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.302653
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.302873
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.301713
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2.301794
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.303112
Train Epoch: 1 [16000/50000 (32%)]	Loss: 2.302608
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.302585
Train Epoch: 1 [22400/50000 (45%)]	Loss: 2.302583
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.302585
Train Epoch: 1 [28800/50000 (58%)]	Loss: 2.302585
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.302585
Train Epoch: 1 [35200/50000 (70%)]	Loss: 2.302585
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.302585
Train Epoch: 1 [41600/50000 (83%)]	Loss: 2.302586
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.302585
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.302585

Test set: Average loss: 0.0187, Accuracy: 901/10000 (9%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 2.302585
Train Epoch: 2 [3200/50000 (6%)]	Loss: 2.302585
Train Epoch: 2 [6400/50000 (13%)]	Loss: 2.302585
Train Ep

Uploading file best.pt: 100%|██████████| 100/100 [00:02<00:00, 42.41it/s]


Train Epoch: 5 [0/50000 (0%)]	Loss: 2.302986
Train Epoch: 5 [3200/50000 (6%)]	Loss: 2.302288
Train Epoch: 5 [6400/50000 (13%)]	Loss: 2.302601
Train Epoch: 5 [9600/50000 (19%)]	Loss: 2.302581
Train Epoch: 5 [12800/50000 (26%)]	Loss: 2.302652
Train Epoch: 5 [16000/50000 (32%)]	Loss: 2.302585
Train Epoch: 5 [19200/50000 (38%)]	Loss: 2.302585
Train Epoch: 5 [22400/50000 (45%)]	Loss: 2.302584
Train Epoch: 5 [25600/50000 (51%)]	Loss: 2.302583
Train Epoch: 5 [28800/50000 (58%)]	Loss: 2.302551
Train Epoch: 5 [32000/50000 (64%)]	Loss: 2.303180
Train Epoch: 5 [35200/50000 (70%)]	Loss: 2.302823
Train Epoch: 5 [38400/50000 (77%)]	Loss: 2.302194
Train Epoch: 5 [41600/50000 (83%)]	Loss: 2.302741
Train Epoch: 5 [44800/50000 (90%)]	Loss: 2.302732
Train Epoch: 5 [48000/50000 (96%)]	Loss: 2.303854

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)

Train Epoch: 6 [0/50000 (0%)]	Loss: 2.303094
Train Epoch: 6 [3200/50000 (6%)]	Loss: 2.303015
Train Epoch: 6 [6400/50000 (13%)]	Loss: 2.302680
Train 

Uploading file best.pt: 100%|██████████| 100/100 [00:02<00:00, 49.36it/s]

In [ ]:
# if in Google Colab, download your model with this
from google.colab import files
files.download("best.pt")